In [1]:
import tmqaalpha2 as tm2
import convex as cx
import requests
import json
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
telegram_chat_id = 0
telegram_bot_token = ""

In [3]:
def send_message(message, telegram_chat_id):
    requests.post("https://api.telegram.org/bot" + str(telegram_bot_token) + "/sendMessage?chat_id="+str(telegram_chat_id)+"&text="+message).json()

def get_updates():
    updates = requests.post("https://api.telegram.org/bot" + str(telegram_bot_token) + "/getUpdates").json()
    return updates

def send_typing(telegram_chat_id):
    requests.post("https://api.telegram.org/bot" + str(telegram_bot_token) + "/sendChatAction?chat_id="+str(telegram_chat_id)+"&action=typing").json()

In [4]:
send_message(
    "Greetings world", 
    telegram_chat_id
)

In [5]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, StringCommandHandler
from telegram import ChatAction
from telegram.error import (TelegramError, Unauthorized, BadRequest, 
                            TimedOut, ChatMigrated, NetworkError)
import requests
import re
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
def get_url():
    contents = requests.get('https://random.dog/woof.json').json()    
    url = contents['url']
    return url

In [7]:
def get_image_url():
    allowed_extension = ['jpg','jpeg','png']
    file_extension = ''
    while file_extension not in allowed_extension:
        url = get_url()
        file_extension = re.search("([^.]*)$",url).group(1).lower()
    return url

In [8]:
def dog(bot, update):
    url = get_image_url()
    chat_id = update.message.chat_id
    bot.send_photo(chat_id=chat_id, photo=url)

In [9]:
def start(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Hi!')

In [10]:
def echo(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=update.message.text)

In [23]:
def ask_tm2(bot, update):
    question = update.message.text
    logger.info("Processing TM2 question: " + str(question))
    bot.send_message(chat_id=update.message.chat_id, text="Rafic -> Thinking about: "+question)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    answer = tm2.answer_question(question)
    if answer:
        top_1 = "Top 1: "+str(tm2.get_wd_label(answer[0][0]))+" ("+str(answer[0][0])+")"
        top_n = "Top n: "+str([(tm2.get_wd_label(h[0]), h[0], h[1]) for i,h in enumerate(answer)])
        logger.info("Top1 for TM2 question: " + str(question) + " --> " + str(top_1))
        bot.send_message(chat_id=update.message.chat_id, text=top_1)
        bot.send_message(chat_id=update.message.chat_id, text=top_n)
    else:
        bot.send_message(chat_id=update.message.chat_id, text="I don't know")
        logger.info("Top1 for Convex question: " + str(question) + " --> I don't know")

In [24]:
def ask_tm2_graph(bot, update, args):
    question = " ".join(args)
    logger.info("Processing TM2 Graph question: " + str(question))
    bot.send_message(chat_id=update.message.chat_id, text="Rafic -> Thinking about: "+question)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    
    q_nlp = tm2.get_nlp(question)
    q_themes = tm2.get_themes(q_nlp)
    q_themes_enhanced = tm2.get_enhanced_themes(q_themes)
    q_predicates = tm2.get_predicates(q_nlp)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    q_predicates = q_predicates if q_predicates[0][1] else tm2.get_predicates_online(q_nlp)
    q_focused_parts = tm2.get_focused_parts_2(q_nlp)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    graph, predicates_dict = tm2.build_graph(q_nlp, q_themes, q_themes_enhanced, q_predicates)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    paths_keywords = tm2.find_paths_keywords(graph, q_nlp, q_themes, q_themes_enhanced, q_predicates, q_focused_parts)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    path_nodes = tm2.find_path_nodes_from_graph_2(graph, paths_keywords)
    paths_nodes_filtered = tm2.paths_nodes_filter(path_nodes)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    hypothesises = tm2.get_hypothesises(q_nlp, paths_keywords, paths_nodes_filtered)
    
    tm2.save_cache_data()
    
    if hypothesises: answer = hypothesises
    else: answer = False

    if answer:
        top_1 = "Top 1: "+str(tm2.get_wd_label(answer[0][0]))+" ("+str(answer[0][0])+")"
        top_n = "Top n: "+str([(tm2.get_wd_label(h[0]), h[0], h[1]) for i,h in enumerate(answer)])
        answer_title = str("Question: \"" + str(question) + "\" --> " + str(top_1))
        logger.info(answer_title)
        bot.send_message(chat_id=update.message.chat_id, text=top_1)
        bot.send_message(chat_id=update.message.chat_id, text=top_n)
    else:
        answer_title = str("Question: \"" + str(question) + "\" --> NO ANSWER")
        bot.send_message(chat_id=update.message.chat_id, text="I don't know")
        logger.info("Top1 for Convex question: " + str(question) + " --> I don't know")
    
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    fig = plt.figure(figsize=(14,14))
    ax = plt.subplot(111)
    ax.set_title(answer_title, fontsize=10)
    #pos = nx.spring_layout(graph)
    labels, colors = tm2.get_elements_from_graph(graph)
    nx.draw(graph, node_size=30, node_color=colors, font_size=10, font_weight='bold', with_labels=True, labels=labels)
    plt.tight_layout()
    file_name = str(str(question)+".png")
    plt.savefig(file_name, format="PNG", dpi = 300)
    plt.clf()
    plt.cla()
    plt.close()
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    bot.send_photo(chat_id=update.message.chat_id, photo=open(file_name, 'rb'))

In [25]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result

In [26]:
def bot_ask_platypus(bot, update, args):
    question = " ".join(args)
    logger.info("Processing Platypus question: "+ str(question))
    bot.send_message(chat_id=update.message.chat_id, text="Platypus -> Thinking about: "+question)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    answer = ask_platypus(question)
    if answer:
        top_1 = str(tm2.get_wd_label(answer))
        logger.info("Top1 for Platypus question: " + str(question) + " --> " + str(top_1))
        bot.send_message(chat_id=update.message.chat_id, text=top_1)
    else:
        bot.send_message(chat_id=update.message.chat_id, text="I don't know")
        logger.info("Top1 for Convex question: " + str(question) + " --> I don't know")

In [27]:
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

In [28]:
def bot_ask_qanswer(bot, update, args):
    question = " ".join(args)
    logger.info("Processing qAnswer question: " + str(question))
    bot.send_message(chat_id=update.message.chat_id, text="qAnswer -> Thinking about: "+question)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    answer = ask_qanswer(question)
    if answer:
        top_1 = str(tm2.get_wd_label(answer))
        logger.info("Top1 for qAnswer question: " + str(question) + " --> " + str(top_1))
        bot.send_message(chat_id=update.message.chat_id, text=top_1)
    else:
        bot.send_message(chat_id=update.message.chat_id, text="I don't know")
        logger.info("Top1 for Convex question: " + str(question) + " --> I don't know")

In [29]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    if not cx_result:
        return cx_result
    if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
        return False
    return cx_result

In [30]:
def bot_ask_convex(bot, update, args):
    question = " ".join(args)
    logger.info("Processing Convex question: "+ str(question))
    bot.send_message(chat_id=update.message.chat_id, text="Convex -> Thinking about: "+question)
    bot.send_chat_action(chat_id=update.message.chat_id, action=ChatAction.TYPING)
    answer = ask_qanswer(question)
    if answer:
        top_1 = str(tm2.get_wd_label(answer))
        logger.info("Top1 for Convex question: " + str(question) + " --> " + str(top_1))
        bot.send_message(chat_id=update.message.chat_id, text=top_1)
    else:
        logger.info("Top1 for Convex question: " + str(question) + " --> I don't know")
        bot.send_message(chat_id=update.message.chat_id, text="I don't know")

In [31]:
def help(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='ask Rafic (TM): type question in the chat')
    bot.send_message(chat_id=update.message.chat_id, text='ask Rafic (TM) with a subgraph: /graph question')
    bot.send_message(chat_id=update.message.chat_id, text='ask Platypus: /platypus question')
    bot.send_message(chat_id=update.message.chat_id, text='ask qAnswer: /qanswer question')
    bot.send_message(chat_id=update.message.chat_id, text='ask Convex: /convex question')

In [32]:
import traceback
def error_callback(bot, update, error):
    try:
        raise error
    except Unauthorized:
        error_info ="Error -> Unauthorized: removed update.message.chat_id from conversation list"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except BadRequest:
        error_info = "Error -> BadRequest: malformed requests"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except TimedOut:
        error_info = "Error -> TimedOut: slow connection problems"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except NetworkError:
        error_info = "Error -> NetworkError: connection problems"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except ChatMigrated as e:
        error_info = "Error -> ChatMigrated: the chat_id of a group has changed, use e.new_chat_id instead"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except TelegramError:
        error_info = "Error -> TelegramError: telegram related errors"
        logger.info(error_info)
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    except:
        error_info = "Your question generated an error, please reformulate or remove special characters."
        logger.info("Error -> "+error_info)
        logger.info(traceback.format_exc())
        bot.send_message(chat_id=update.message.chat_id, text=error_info)
    
        
    #logger.info("Error: " + str(update.message.text))
    #logger.warning('Update "%s" caused error "%s"', bot, update.error)

In [ ]:
updater = Updater(telegram_bot_token)
dp = updater.dispatcher
dp.add_handler(CommandHandler('dog',dog))
dp.add_handler(CommandHandler('help',help))
dp.add_handler(CommandHandler("platypus", bot_ask_platypus, pass_args=True))
dp.add_handler(CommandHandler("convex", bot_ask_convex, pass_args=True))
dp.add_handler(CommandHandler("qanswer", bot_ask_qanswer, pass_args=True))
dp.add_handler(CommandHandler("graph", ask_tm2_graph, pass_args=True))
dp.add_handler(MessageHandler(Filters.text, ask_tm2))
dp.add_error_handler(error_callback)
updater.start_polling()
updater.idle()

2019-12-11 22:52:07,558 - __main__ - INFO - Processing TM2 question: lol
2019-12-11 22:52:07,868 - __main__ - INFO - Top1 for Convex question: lol --> I don't know
